Для определения лучшего набора акционных предложений, стоит проанализировать несколько ключевых метрик, таких как ARPU (Average Revenue Per User), CR (Conversion Rate) и ARPPU (Average Revenue Per Paying User).

Рассчитаем CR (Conversion Rate):

In [1]:
# Данные
n_a = 202103
n_b = 202667
payers_a = 1928
payers_b = 1805

# Конверсия
CR_a = payers_a / n_a
CR_b = payers_b / n_b

print(f"Конверсия в покупку (группа a): {CR_a:.4f}")
print(f"Конверсия в покупку (группа b): {CR_b:.4f}")

Конверсия в покупку (группа a): 0.0095
Конверсия в покупку (группа b): 0.0089


ARPU в двух группах отличается на 5 %. Но мы не знаем, случайно ли это отличие или нет.

В таком случае, для определения статистической значимости между двумя группами, мы можем провести z-тест или t-тест (например, независимый двухвыборочный t-тест). В данной ситуации мы сравниваем между собой две независимые выборки, поэтому подходит независимый двухвыборочный t-тест.

Загрузим данные и подготовим их для анализа:

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-a-30/final_project/Проект_1_Задание_2.csv', sep=';')

revenue_control = data[data['testgroup'] == 'a']['revenue']
revenue_test = data[data['testgroup'] == 'b']['revenue']

Рассчитаем среднее значение и стандартное отклонение для каждой группы:

In [3]:
mean_control = np.mean(revenue_control)
mean_test = np.mean(revenue_test)

std_control = np.std(revenue_control, ddof=1)
std_test = np.std(revenue_test, ddof=1)

Проведем независимый двухвыборочный t-тест:

In [4]:
t_stat, p_value = stats.ttest_ind(revenue_control, revenue_test, equal_var=False)
print(f't-statistic: {t_stat}')
print(f'p-value: {p_value}')

t-statistic: -0.6234876562792212
p-value: 0.5329647014130866


В данном случае t-статистика равна -0.6234, что означает, что среднее значение ARPU в контрольной группе ниже, чем в тестовой группе, и разница составляет примерно 0.6235 стандартных отклонений.

Результаты показывают, что p-value составляет 0.5329, что больше обычного уровня значимости 0.05. Это означает, что мы не можем отклонить нулевую гипотезу о равенстве средних двух групп. В данной ситуации различия в ARPU между контрольной и тестовой группой не являются статистически значимыми.

В связи с этим, мы не можем сделать однозначный вывод о том, какой набор акционных предложений является лучшим.

Можно провести z-тест для сравнения конверсий двух групп, используя уже полученные значения.

Используя ранее полученные значения конверсии для каждой группы, проведем z-тест:

In [5]:
from scipy.stats import norm

# Данные
n_a = 202103
n_b = 202667
payers_a = 1928
payers_b = 1805

# Конверсия
CR_a = payers_a / n_a
CR_b = payers_b / n_b

# z-тест
pooled_prob = (payers_a + payers_b) / (n_a + n_b)
z_score = (CR_b - CR_a) / np.sqrt(pooled_prob * (1 - pooled_prob) * (1 / n_a + 1 / n_b))
p_value = norm.sf(abs(z_score)) * 2  # Двусторонний тест

print(f"z-score: {z_score:.4f}")
print(f"p-value: {p_value:.4f}")

z-score: -2.1080
p-value: 0.0350


В данном случае, z-статистика равна 2.108, что означает, что разница между конверсиями двух групп составляет примерно 2.1 стандартных отклонений.

В данном случае p-value равно 0.035, что меньше стандартного порога значимости 0.05.

Таким образом, мы можем сделать вывод, что различия между конверсиями в платящих для контрольной и тестовой групп статистически значимы. Поскольку z-статистика положительна, это означает, что конверсия в платящих выше в контрольной группе по сравнению с тестовой группой. Следовательно, на основе этой метрики можно предположить, что набор акционных предложений в контрольной группе оказался лучше для привлечения платящих пользователей.

Рассчитаем ARPPU (Average Revenue Per Paying User) для контрольной и тестовой групп. ARPPU — это средний доход от каждого платящего пользователя.

Сначала найдем сумму доходов от платящих пользователей в каждой группе, затем поделим их на количество платящих пользователей.

In [6]:
paying_users_control = (revenue_control > 0).sum()
paying_users_test = (revenue_test > 0).sum()

total_revenue_control = revenue_control[revenue_control > 0].sum()
total_revenue_test = revenue_test[revenue_test > 0].sum()

ARPPU_control = total_revenue_control / paying_users_control
ARPPU_test = total_revenue_test / paying_users_test

print(f'ARPPU (Control group): {ARPPU_control:.2f}')
print(f'ARPPU (Test group): {ARPPU_test:.2f}')

ARPPU (Control group): 2664.00
ARPPU (Test group): 3003.66


Теперь у нас есть значения ARPPU для контрольной и тестовой групп. ARPPU (Average Revenue Per Paying User) — это средний доход от каждого платящего пользователя.

Согласно этим результатам, ARPPU для тестовой группы выше, чем для контрольной группы. Это может указывать на то, что акционные предложения в тестовой группе оказались более привлекательными для платящих пользователей, что привело к увеличению среднего дохода с каждого из них.

Однако, прежде чем делать окончательный вывод, может быть полезно проверить статистическую значимость различий между ARPPU контрольной и тестовой групп с использованием t-теста или z-теста, как мы делали ранее для других метрик. Это поможет убедиться, что разница между значениями ARPPU не является случайной и действительно связана с разными акционными предложениями.

Для проверки статистической значимости различий между ARPPU контрольной и тестовой групп, мы можем использовать t-тест. В данном случае мы сравниваем среднее значение дохода от платящих пользователей в двух группах.

Сначала мы создадим две новые выборки, содержащие только платящих пользователей из каждой группы:

In [7]:
paying_revenue_control = revenue_control[revenue_control > 0]
paying_revenue_test = revenue_test[revenue_test > 0]

Теперь мы можем провести t-тест для этих двух выборок:

In [8]:
t_stat, p_value = stats.ttest_ind(paying_revenue_control, paying_revenue_test, equal_var=False)
print(f't-statistic: {t_stat}')
print(f'p-value: {p_value}')

t-statistic: -1.6446302664300838
p-value: 0.10020780398916015


Таким образом, хотя ARPPU тестовой группы выше, чем контрольной группы, различия между ними не достаточно весомые, чтобы делать вывод о том, что акционные предложения в тестовой группе действительно эффективнее.

Исходя из расчетов, которые мы провели, можно сделать следующие выводы:

Конверсия в платящих пользователей (CR) выше в контрольной группе по сравнению с тестовой группой. Различия в конверсии статистически значимы, что указывает на то, что акционные предложения в контрольной группе привлекли больше платящих пользователей.

Средний доход от каждого платящего пользователя (ARPPU) выше в тестовой группе по сравнению с контрольной группой. Однако статистически значимых различий между этими значениями обнаружено не было, поэтому мы не можем с уверенностью сказать, что акционные предложения в тестовой группе привели к увеличению среднего дохода от каждого платящего пользователя.

Таким образом, основываясь на имеющихся данных и расчетах, можно сделать вывод, что акционные предложения в контрольной группе были более эффективными в привлечении платящих пользователей. Однако, поскольку разница в ARPPU между группами не является статистически значимой, мы не можем утверждать, что одна из групп превосходит другую в смысле среднего дохода от каждого платящего пользователя.